<a href="https://colab.research.google.com/github/HilaShoshan/MusicGenreClassification/blob/main/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description

**My Data**

The data contains music from the following genres:

*blues, classical, country, disco, hiphop, jazz, metal, pop, reggae and rock.* 

There are several data bases that refer to the same music, and I'm using each one as needed. 

**My Questions**

I want to know if it is possible to classify a song for its genre based on an audio segment from it. 

More specific sub-questions are: 


1.   What is the technique that will bring the best results?

2.   Will using the techniques we learned with the data from the CSV files
only yield good results?

3. Will deep learning techniques, like CNN on the pictures data will predict
the genre in a good way?

4. Are there types of songs that a model would have a very hard time
distinguishing between?

5. Do genres that are not at all close to each other still have the same
characteristics (for example in a certain part of the music)?

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read DataBases

**Images dataset**

A visual representation for each audio file. The audio files were converted to Mel Spectrograms.

**CSV dataset**

Containing features of the audio files. One file has for each song (30 seconds long) a mean and variance computed over multiple features that can be extracted from an audio file. The other file has the same structure, but the songs were split before into 3 seconds audio files (this way increasing 10 times the amount of data we fuel into our classification models).

I'll read the 30-seconds-features for start. 

In [26]:
features_df = pd.read_csv("drive/MyDrive/Colab Notebooks/Data/features_30_sec.csv")
features_df.head(10)

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,1.297741e+05,2002.449060,85882.761315,3805.839606,9.015054e+05,0.083045,0.000767,-4.529724e-05,0.008172,0.000008,0.005698,123.046875,-113.570648,2564.207520,121.571793,295.913818,-19.168142,235.574432,42.366421,151.106873,-6.364664,167.934799,18.623499,89.180840,-13.704891,67.660492,15.343150,68.932579,-12.274110,82.204201,10.976572,63.386311,-8.326573,61.773094,8.803792,51.244125,-3.672300,41.217415,5.747995,40.554478,-5.162882,49.775421,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,3.758501e+05,2039.036516,213843.755497,3550.522098,2.977893e+06,0.056040,0.001448,1.395807e-04,0.005099,-0.000178,0.003063,67.999589,-207.501694,7764.555176,123.991264,560.259949,8.955127,572.810913,35.877647,264.506104,2.907320,279.932922,21.510466,156.477097,-8.560436,200.849182,23.370686,142.555954,-10.099661,166.108521,11.900497,104.358612,-5.555639,105.173630,5.376327,96.197212,-2.231760,64.914291,4.220140,73.152534,-6.012148,52.422142,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,1.564676e+05,1747.702312,76254.192257,3042.260232,7.840345e+05,0.076291,0.001007,2.105576e-06,0.016342,-0.000019,0.007458,161.499023,-90.722595,3319.044922,140.446304,508.765045,-29.093889,411.781219,31.684334,144.090317,-13.984504,155.493759,25.764742,74.548401,-13.664875,106.981827,11.639934,106.574875,-11.783643,65.447945,9.718760,67.908859,-13.133803,57.781425,5.791199,64.480209,-8.907628,60.385151,-1.077000,57.711136,-9.229274,36.580986,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,1.843559e+05,1596.412872,166441.494769,2184.745799,1.493194e+06,0.033309,0.000423,4.583644e-07,0.019054,-0.000014,0.002712,63.024009,-199.544205,5507.517090,150.090897,456.505402,5.662678,257.161163,26.859079,158.267303,1.771399,268.034393,14.234031,126.794128,-4.832006,155.912079,9.286494,81.273743,-0.759186,92.114090,8.137607,71.314079,-3.200653,110.236687,6.079319,48.251999,-2.480174,56.799400,-1.079305,62.289902,-2.870789,51.651592,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,3.433999e+05,1748.172116,88445.209036,3579.757627,1.572978e+06,0.101461,0.001954,-1.756129e-05,0.004814,-0.000010,0.003094,135.999178,-160.337708,5195.291992,126.219635,853.784729,-35.587811,333.792938,22.148071,193.456100,-32.478600,336.276825,10.852294,134.831573,-23.352329,93.257095,0.498434,124.672127,-11.793437,130.073349,1.207256,99.675575,-13.088418,80.254066,-2.813867,86.430626,-6.933385,89.555443,-7.552725,70.943336,-9.164666,75.793404,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
5,blues.00005.wav,661794,0.302456,0.087532,0.103494,0.003981,1831.993940,1.030482e+06,1729.653287,201910.508633,3481.517592,3.274440e+06,0.094042,0.0062

In [27]:
print(features_df.shape, '\n')

print(features_df.isnull().sum())

(1000, 60) 

filename                   0
length                     0
chroma_stft_mean           0
chroma_stft_var            0
rms_mean                   0
rms_var                    0
spectral_centroid_mean     0
spectral_centroid_var      0
spectral_bandwidth_mean    0
spectral_bandwidth_var     0
rolloff_mean               0
rolloff_var                0
zero_crossing_rate_mean    0
zero_crossing_rate_var     0
harmony_mean               0
harmony_var                0
perceptr_mean              0
perceptr_var               0
tempo                      0
mfcc1_mean                 0
mfcc1_var                  0
mfcc2_mean                 0
mfcc2_var                  0
mfcc3_mean                 0
mfcc3_var                  0
mfcc4_mean                 0
mfcc4_var                  0
mfcc5_mean                 0
mfcc5_var                  0
mfcc6_mean                 0
mfcc6_var                  0
mfcc7_mean                 0
mfcc7_var                  0
mfcc8_mean                 0
m

In [28]:
features_df.describe()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,harmony_mean,harmony_var,perceptr_mean,perceptr_var,tempo,mfcc1_mean,mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean,mfcc3_var,mfcc4_mean,mfcc4_var,mfcc5_mean,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc7_mean,mfcc7_var,mfcc8_mean,mfcc8_var,mfcc9_mean,mfcc9_var,mfcc10_mean,mfcc10_var,mfcc11_mean,mfcc11_var,mfcc12_mean,mfcc12_var,mfcc13_mean,mfcc13_var,mfcc14_mean,mfcc14_var,mfcc15_mean,mfcc15_var,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,662030.846000,0.378682,0.086340,0.130930,0.003051,2201.780898,4.696916e+05,2242.541070,137079.155165,4571.549304,1.844345e+06,0.103612,0.002986,-3.659332e-04,0.012540,-0.000395,5.670073e-03,119.505363,-144.472987,3747.875414,99.554278,706.899539,-8.920291,468.286436,36.292447,220.447219,-1.144840,174.404462,14.633491,127.232071,-5.129038,116.367019,10.119172,88.198988,-6.994062,88.566933,7.729785,81.733490,-6.021069,75.551228,4.471693,68.670817,-4.795884,67.871712,1.781900,64.764096,-3.869308,62.873761,1.148144,60.730958,-3.966028,62.633624,0.507696,63.712586,-2.328761,66.231930,-1.095348,70.126096
std,1784.073992,0.081705,0.007735,0.065683,0.003634,715.960600,4.008995e+05,526.316473,96455.666326,1574.791602,1.425085e+06,0.041821,0.003024,1.683946e-03,0.011653,0.001082,6.500325e-03,28.264176,100.229406,2770.558394,31.330873,439.120256,21.690676,287.143462,16.664665,116.236941,12.221340,100.767823,11.839591,68.475115,9.940460,58.400048,10.463270,41.048213,8.283921,39.494072,7.937309,36.733649,6.817013,38.135183,6.715787,32.625534,6.170181,33.154276,5.008385,34.450290,4.873599,33.904101,4.578948,33.781951,4.549697,33.479172,3.869105,34.401977,3.755957,37.174631,3.837007,45.228512
min,660000.000000,0.171939,0.044555,0.005276,0.000004,570.040355,7.911251e+03,898.066208,10787.185064,749.140636,1.468612e+04,0.021697,0.000044,-2.606458e-02,0.000049,-0.007150,8.299008e-07,54.978391,-552.120056,174.571365,-1.461542,93.104752,-89.870193,35.467754,-18.727547,34.263149,-38.890347,29.912008,-28.415911,20.847921,-32.930431,29.027332,-24.915823,22.464230,-31.640574,25.825863,-12.050177,19.273201,-28.057501,21.402756,-15.796446,18.642517,-27.523129,17.293922,-12.594489,14.250607,-17.539125,9.769691,-15.693844,9.169314,-17.234728,13.931521,-11.963694,15.420555,-18.501955,13.487622,-19.929634,7.956583
25%,661504.000000,0.319562,0.082298,0.086657,0.000942,1627.697311,1.843505e+05,1907.240605,67376.554428,3380.069642,7.727312e+05,0.070244,0.001079,-1.041584e-04,0.004383,-0.000537,1.292592e-03,99.384014,-200.763893,1844.488129,76.779915,398.020767,-24.219811,270.466576,24.108212,129.397377,-9.979157,101.266209,5.098573,76.505194,-12.870810,73.546917,1.609181,57.349041,-13.246939,59.882313,1.620753,55.734016,-10.964135,51.242420,-0.547371,46.854038,-9.357906,46.294396,-1.639417,43.416593,-7.165884,42.236251,-1.863280,40.376442,-7.207225,40.830875,-2.007015,41.884240,-4.662925,41.710184,-3.368996,42.372865
50%,661794.000000,0.383148,0.086615,0.122443,0.001816,2209.263090,3.384862e+05,2221.392843,111977.548036,4658.524473,1.476115e+06,0.099503,0

# Arrange Dataset

In [29]:
del features_df['filename']
print(features_df.shape)

(1000, 59)


In [30]:
# all the columns on the csv file are numerical except from the label column 
# I'll use one label encoding to make it numeric
"""
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

features_df['label_encode'] = labelencoder.fit_transform(features_df['label'])

dont need to encode the label column also ... 
"""

"\nfrom sklearn.preprocessing import LabelEncoder\n\nlabelencoder = LabelEncoder()\n\nfeatures_df['label_encode'] = labelencoder.fit_transform(features_df['label'])\n\ndont know if we should encode the label column also ... \n"

In [31]:
# Split the data into X (features dataframe) and y (the label column)

X = features_df.iloc[:, 0:-1]
y = features_df[['label']]
print(X.shape)
print(y.shape, '\n')

(1000, 58)
(1000, 1) 



In [32]:
# Now I'll normalize the X data: 

from sklearn import preprocessing

x = X.values  # returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_norm = pd.DataFrame(x_scaled)

X_norm.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57
0,0.113487,0.362279,0.695468,0.318188,0.101983,0.314117,0.040233,0.422879,0.109789,0.385532,0.102567,0.242545,0.025209,0.632779,0.094883,0.553280,0.120083,0.378307,0.738100,0.085135,0.632371,0.051568,0.482433,0.069266,0.614443,0.134574,0.462466,0.130984,0.639217,0.139176,0.351102,0.103138,0.544576,0.128074,0.381428,0.192710,0.586480,0.131694,0.433846,0.070939,0.633516,0.117862,0.587829,0.087993,0.596097,0.054619,0.413989,0.096032,0.564186,0.112704,0.541287,0.057615,0.422444,0.082450,0.487950,0.109503,0.599189,0.078255
1,0.113487,0.343622,0.793392,0.230894,0.085580,0.248405,0.121475,0.436889,0.296867,0.353329,0.342717,0.135778,0.048944,0.637275,0.058988,0.538949,0.064545,0.072368,0.580010,0.270406,0.644806,0.118782,0.674332,0.185998,0.549183,0.265181,0.594300,0.237271,0.678448,0.276241,0.445051,0.458711,0.653163,0.330990,0.424254,0.479508,0.610012,0.254011,0.494774,0.147203,0.545250,0.280378,0.623332,0.175153,0.546444,0.122308,0.385580,0.102385,0.570198,0.120353,0.574685,0.118286,0.448184,0.102997,0.548800,0.099081,0.579640,0.075947
2,0.113487,0.389832,0.640692,0.433652,0.099064,0.254261,0.049046,0.325334,0.095712,0.289224,0.088981,0.215844,0.033575,0.633932,0.190314,0.551173,0.157167,0.592014,0.776555,0.112027,0.729382,0.105689,0.414705,0.130259,0.507010,0.126493,0.354124,0.119178,0.736259,0.109374,0.351832,0.208114,0.494483,0.231821,0.391088,0.135435,0.554444,0.145195,0.328144,0.063925,0.555934,0.165714,0.458799,0.158494,0.374297,0.090245,0.277967,0.064359,0.622467,0.081898,0.331003,0.085962,0.370988,0.116420,0.453724,0.087317,0.501381,0.045405
3,0.113487,0.473508,0.777954,0.345856,0.229160,0.129376,0.058253,0.267404,0.227566,0.181068,0.171001,0.045909,0.013206,0.633892,0.221999,0.551558,0.057147,0.044715,0.593403,0.189995,0.778954,0.092401,0.651866,0.076738,0.458480,0.142821,0.578149,0.225979,0.579569,0.215785,0.513140,0.338743,0.462649,0.162088,0.608218,0.226584,0.514173,0.155361,0.546556,0.156100,0.563354,0.107045,0.617210,0.145306,0.374222,0.099752,0.490660,0.100536,0.565151,0.091876,0.484549,0.092509,0.460684,0.068975,0.538690,0.062612,0.468060,0.048168
4,0.113487,0.277759,0.681062,0.219641,0.083075,0.327270,0.110761,0.325514,0.113536,0.357017,0.180228,0.315353,0.066611,0.633454,0.055655,0.551896,0.065193,0.450292,0.659389,0.178872,0.656260,0.193416,0.370394,0.103264,0.411100,0.183350,0.091165,0.290741,0.533615,0.232155,0.174918,0.171473,0.343774,0.281700,0.390895,0.356334,0.337660,0.240030,0.329142,0.103414,0.334333,0.245069,0.507456,0.265786,0.163848,0.117721,0.280128,0.158487,0.383289,0.200462,0.410225,0.156424,0.403875,0.120300,0.424652,0.130173,0.233047,0.094836
5,0.113487,0.265416,0.676207,0.250109,0.143674,0.326491,0.337601,0.318423,0.279421,0.344626,0.377015,0.286022,0.215842,0.633886,0.093836,0.550629,0.068315,0.082583,0.630043,0.254119,0.615068,0.788765,0.493366,0.214231,0.497737,0.519994,0.243741,0.382921,0.548137,0.500108,0.222789,0.375663,0.380136,0.306684,0.454066,0.500430,0.283153,0.241375,0.358078,0.096899,0.344841,0.217150,0.448052,0.231707,0.086389,0.069150,0.456099,0.120746,0.347064,0.165154,0.536132,0.139675,0.335620,0.085128,0.465527,0.069326,0.334841,0.077000
6,0.113487,0.242786,0.777675,0.347845,0.317934,0.230085,0.141947,0.187987,0.254732,0.258115,0.185833,0.203038,0.065044,0.633646,0.197105,0.550949,0.125474,0.093364,0.609378,0.337811,0.677176,0.467773,0.365060,0.440303,0.520534,0.513781,0.710912,1.000000,0.377914,0.529410,0.220217,0.329001,0.340725,0.221737,0.495424,0.359315,0.166403,0.400568,0.444551,0.116043,0.326228,0.164793,0.515766,0.227425,0.085929,0.101622,0.368085,0.183898,0.192988,0.193033,0.220128,0.109583,0.182522,0.071850,0.453821,0.091521,0.380919,0.124900
7,0.113487,0.276598,0.760707,0.322248,0.199704,0.228093,0.145813,0.260074,0.230154,0.278197,0.186723,0.157137,0.062939,0.634353,0.153882,0.548958,0.101967,0.207341,0.6

# train test split

In [39]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, shuffle=True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(700, 58)
(300, 58)
(700, 1)
(300, 1)


# Machine Learning Techniques

## SVC

---- explain 

In [40]:
# Import Support Vector Classifier
from sklearn.svm import SVC

# Create a svm Classifier
svc = SVC(probability=True, kernel='linear')  # Linear Kernel

# Train the model using the training sets
model = svc.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.71


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Adaboost

---- explain 

In [41]:
from sklearn.ensemble import AdaBoostClassifier

# Create adaboost classifier object
# use the default DecisionTreeClassifier as base_estimator

adaboost = AdaBoostClassifier(n_estimators=50, learning_rate=1)

# Train Adaboost Classifer
model = adaboost.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.2833333333333333


In [43]:
# use SVC as base_estimator

# Create adaboost classifier object
adaboost = AdaBoostClassifier(n_estimators=50, base_estimator=svc, learning_rate=1)

# Train Adaboost Classifer
model = adaboost.fit(X_train, y_train)

# Predict the response for test dataset
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.56


# KNN

-- explain